In [26]:
# import pandas as pd
import dask.dataframe as dd

In [14]:
# Define the column headers
predication_headers = [
    'PREDICATION_ID', 'SENTENCE_ID', 'PMID', 'PREDICATE', 'SUBJECT_CUI',
    'SUBJECT_NAME', 'SUBJECT_SEMTYPE', 'SUBJECT_NOVELTY', 'OBJECT_CUI',
    'OBJECT_NAME', 'OBJECT_SEMTYPE', 'OBJECT_NOVELTY', 'FACT_VALUE_CHAR',
    'MOD_SCALE_CHAR', 'MOD_VALUE_FLOAT'
]

predication_aux_headers = [
    'PREDICATION_AUX_ID', 'PREDICATION_ID', 'SUBJECT_TEXT', 'SUBJECT_DIST',
    'SUBJECT_MAXDIST', 'SUBJECT_START_INDEX', 'SUBJECT_END_INDEX', 'SUBJECT_SCORE',
    'INDICATOR_TYPE', 'PREDICATE_START_INDEX', 'PREDICATE_END_INDEX', 'OBJECT_TEXT',
    'OBJECT_DIST', 'OBJECT_MAXDIST', 'OBJECT_START_INDEX', 'OBJECT_END_INDEX',
    'OBJECT_SCORE', 'CURR_TIMESTAMP'
]

predication_dtype = {
    'PREDICATION_ID': 'int32',
    'SENTENCE_ID': 'int32',
    'PMID': 'str',
    'PREDICATE': 'str',
    'SUBJECT_CUI': 'str',
    'SUBJECT_NAME': 'str',
    'SUBJECT_SEMTYPE': 'str',
    'SUBJECT_NOVELTY': 'int8',
    'OBJECT_CUI': 'str',
    'OBJECT_NAME': 'str',
    'OBJECT_SEMTYPE': 'str',
    'OBJECT_NOVELTY': 'int8',
    'FACT_VALUE_CHAR': 'str',
    'MOD_SCALE_CHAR': 'str',
    'MOD_VALUE_FLOAT': 'float32'
}

predication_aux_dtype = {
    'PREDICATION_AUX_ID': 'int32',
    'PREDICATION_ID': 'int32',
    'SUBJECT_TEXT': 'str',
    'SUBJECT_DIST': 'int32',
    'SUBJECT_MAXDIST': 'int32',
    'SUBJECT_START_INDEX': 'int32',
    'SUBJECT_END_INDEX': 'int32',
    'SUBJECT_SCORE': 'int32',
    'INDICATOR_TYPE': 'str',
    'PREDICATE_START_INDEX': 'int32',
    'PREDICATE_END_INDEX': 'int32',
    'OBJECT_TEXT': 'str',
    'OBJECT_DIST': 'int32',
    'OBJECT_MAXDIST': 'int32',
    'OBJECT_START_INDEX': 'int32',
    'OBJECT_END_INDEX': 'int32',
    'OBJECT_SCORE': 'int32',
    'CURR_TIMESTAMP': 'string'  # Assuming timestamp is read as string
}

In [ ]:
# Read the CSV file with the specified headers using Dask
df = dd.read_csv('semmed_data/predication.csv', names=predication_headers, dtype=predication_dtype, encoding='ISO-8859-1', on_bad_lines='skip', assume_missing=True, na_values=['\\N'])

# Read the CSV file with the specified headers using Dask
df_aux = dd.read_csv('semmed_data/predication_aux.csv', names=predication_aux_headers, dtype=predication_aux_dtype, encoding='ISO-8859-1', on_bad_lines='skip', assume_missing=True, na_values=['\\N'], parse_dates=['CURR_TIMESTAMP']) 

In [ ]:
df.head(5)

In [ ]:
df_aux.head(5)

In [ ]:
# Export both DataFrames to a single .pkl file
# dataframes = {'df': df, 'df_aux': df_aux}
# dd.to_pickle(dataframes, 'semmed_data/dataframes.pkl')

In [18]:
# Merge with Dask
merged_df = dd.merge(df, df_aux, on='PREDICATION_ID', how='inner', indicator=True)

In [ ]:
merged_df.head(5)

In [29]:
def fill_na_in_integer_columns(df, columns):
    for col in columns:
        na_count = df[col].isnull().sum().compute()
        if na_count > 0:
            print(f"Column {col} has {na_count} NA values.")
            # Fill NA values with -1
            df[col] = df[col].fillna(-1)

In [43]:
# ... existing code ...
# Read the CSV file with the specified headers and dtypes using Dask
df = dd.read_csv(
    'semmed_data/predication.csv',
    names=predication_headers,
    dtype=predication_dtype,
    on_bad_lines='skip',
    assume_missing=True,
    na_values=['\\N'], 
    na_filter=False
)

# Read the CSV file with the specified headers and dtypes using Dask
df_aux = dd.read_csv(
    'semmed_data/predication_aux.csv',
    names=predication_aux_headers,
    dtype=predication_aux_dtype,
    on_bad_lines='skip',
    assume_missing=True,
    na_values=['\\N'],
    na_filter=False, 
    parse_dates=['CURR_TIMESTAMP']  # Specify the column to parse as datetime
)

# # Proceed with merging if no issues are found
# merged_df = dd.merge(df, df_aux, on='PREDICATION_ID', how='inner', indicator=True)

# # Check the first few rows of the merged DataFrame
# merged_df.head(5)

ValueError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: could not convert string to float: '\\N'

In [33]:
# Function to fill NA values in integer columns aggressively
def fill_na_in_integer_columns(df, columns):
    for col in columns:
        try:
            na_count = df[col].isnull().sum().compute()
            if na_count > 0:
                print(f"Column {col} has {na_count} NA values. Filling with -1.")
                # Fill NA values with -1
                df[col] = df[col].fillna(-1)
        except ValueError as e:
            print(f"Error processing column {col}: {e}. Attempting to fill NA values.")
            # Attempt to fill NA values directly
            df[col] = df[col].fillna(-1)

# Check and fill NA values for df
fill_na_in_integer_columns(df, predication_headers)

# Check and fill NA values for df_aux, including CURR_TIMESTAMP
fill_na_in_integer_columns(df_aux, predication_aux_headers)

Error processing column PREDICATION_ID: Integer column has NA values in column 11. Attempting to fill NA values.
Error processing column SENTENCE_ID: Integer column has NA values in column 11. Attempting to fill NA values.
Error processing column PMID: Integer column has NA values in column 11. Attempting to fill NA values.
Error processing column PREDICATE: Integer column has NA values in column 11. Attempting to fill NA values.
Error processing column SUBJECT_CUI: Integer column has NA values in column 11. Attempting to fill NA values.
Error processing column SUBJECT_NAME: Integer column has NA values in column 11. Attempting to fill NA values.
Error processing column SUBJECT_SEMTYPE: Integer column has NA values in column 11. Attempting to fill NA values.
Error processing column SUBJECT_NOVELTY: Integer column has NA values in column 11. Attempting to fill NA values.
Error processing column OBJECT_CUI: Integer column has NA values in column 11. Attempting to fill NA values.
Error pr

In [20]:
dataframes = [
    merged_df[['SUBJECT_CUI', 'SUBJECT_NAME', 'SUBJECT_SEMTYPE', 'SUBJECT_NOVELTY']].rename(
        columns={
            'SUBJECT_CUI': 'CUI',
            'SUBJECT_NAME': 'NAME',
            'SUBJECT_SEMTYPE': 'SEMTYPE',
            'SUBJECT_NOVELTY': 'NOVELTY'
        }
    ),
    merged_df[['OBJECT_CUI', 'OBJECT_NAME', 'OBJECT_SEMTYPE', 'OBJECT_NOVELTY']].rename(
        columns={
            'OBJECT_CUI': 'CUI',
            'OBJECT_NAME': 'NAME',
            'OBJECT_SEMTYPE': 'SEMTYPE',
            'OBJECT_NOVELTY': 'NOVELTY'
        }
    )
]

# Concatenate both DataFrames in one step
concept = dd.concat(dataframes, ignore_index=True)

# Drop duplicates to ensure uniqueness
concept = concept.drop_duplicates(subset=['CUI']).reset_index(drop=True)

In [ ]:
concept

In [41]:
df.compute()


ValueError: Integer column has NA values in column 11